In [1]:
%run ~/relmapping/annot/notebooks/annot__init__.ipynb
annot_ = 'annot_ce10'
def mp(fp, annot_=annot_): return os.path.join(annot_, 'canonical_geneset', fp)

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [2]:
# canonical_geneset.gtf.gz: WS260 canonical geneset in ce10 coordinates

# Download WS260 annotations:
# !cd .wget; wget -m --no-parent ftp://ftp.wormbase.org/pub/wormbase/releases/WS260/

def liftover_WS235_WS220_gtf(chroms, starts, ends, strands): # Liftover wrapper
    """
    # http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4512556/
    # note that while annotations changed, the N2 genome sequence remained identical from WS215 through WS234
    Test: 
    list(liftover_WS230_WS220(
    chroms=['chrI', 'chrX', 'chrIII', 'chrV', 'chrIV', 'chrII'],
    starts=[1,1,4,1,1, 1],
    ends=[12507921,13713648,471811,15465174,11718826, 11718826],
    strands=['+', '-', '+', '-', '+']))
    """
    fp_inp = mp('liftover_fp_inp.tsv')
    fp_out = mp('liftover_fp_out.tsv')
    with open(fp_inp, 'w') as fh_inp:
        for (chrom, start, end, strand) in zip(chroms, starts, ends, strands):
            print('\t'.join(map(str, (chrom, start, end, strand))), file=fh_inp)
            #print('%s\t%s' % (chrom, start), file=fh_inp)

    liftover_bin = '/mnt/home1/ahringer/jj374/lab/raw_data/datastore/Kenneth_Evans/ConvertWormAssembly-1-0-0/ConvertWormAssembly.1.0.0.pl'
    liftover_dir = '/mnt/home1/ahringer/jj374/lab/raw_data/datastore/Kenneth_Evans/ConvertWormAssembly-1-0-0/CHROMOSOME-DIFFERENCES-to-WS244'
    !{liftover_bin} \
        --input={fp_inp} \
        --output={fp_out} \
        --backwards \
        --release1=WS235 \
        --release2=WS220 \
        --chrfield=1 \
        --posfield=2 \
        --posfield=3 \
        --postype=1 \
        --postype=1 \
        --strandfield=4 \
        --refdir={liftover_dir}
    
    df_out = pd.read_csv(fp_out, sep='\t', names=['chrom', 'start', 'end', 'strand'])
    for i, r in df_out.iterrows():
        yield((r['chrom'], r['start'], r['end'], r['strand']))
    
    !rm {fp_inp}
    !rm {fp_out}

def to_ce10(df_inp): # Use liftover wrapper on a gtf-DataFrame
    df_out = df_inp.query('(chrom != "CHROMOSOME_MtDNA") & (chrom != "MtDNA")').reset_index(drop=True).copy()
    df_WS220 = pd.DataFrame.from_records(liftover_WS235_WS220_gtf(df_out['chrom'], df_out['start'], df_out['end'], df_out['strand']), columns=['chrom', 'start', 'end', 'strand'])
    df_out['chrom'] = [*map(lambda chrom: 'chr' + chrom.lstrip('CHROMOSOME_'), df_WS220['chrom'])]
    df_out['start'] = df_WS220['start'].copy()
    df_out['end'] = df_WS220['end'].copy()
    df_out['strand'] = df_WS220['strand'].copy()
    return df_out.sort_values(['chrom', 'start', 'end', 'strand']).reset_index(drop=True)

# Genes & transcripts from the canonical geneset
kwargs_to_gtf = {'index': False, 'header': False, 'quoting': csv.QUOTE_NONE, 'sep': '\t'}
fp_ = 'wget/ftp.wormbase.org/pub/wormbase/releases/WS260/species/c_elegans/PRJNA13758/c_elegans.PRJNA13758.WS260.canonical_geneset.gtf.gz'
df_ = pd.read_csv(fp_, sep='\t', names=yp.NAMES_GTF, comment='#')
df_ = to_ce10(df_)
df_.head()

Input file  = annot_ce10/canonical_geneset/liftover_fp_inp.tsv 
Output file = annot_ce10/canonical_geneset/liftover_fp_out.tsv 
Lines processed from input file              = 726282
Comment / blank lines written to output file = 0
Data lines written to output file            = 726282
Number of records changed                    = 699702


,chrom,source,feature,start,end,score,strand,frame,attribute
0,chrI,WormBase,gene,3747,3909,.,-,.,"gene_id ""WBGene00023193""; gene_source ""WormBas..."
1,chrI,WormBase,transcript,3747,3909,.,-,.,"gene_id ""WBGene00023193""; transcript_id ""Y74C9..."
2,chrI,WormBase,exon,3747,3909,.,-,.,"gene_id ""WBGene00023193""; transcript_id ""Y74C9..."
3,chrI,WormBase,three_prime_utr,4116,4220,.,-,.,"gene_id ""WBGene00022277""; transcript_id ""Y74C9..."
4,chrI,WormBase,exon,4116,4358,.,-,.,"gene_id ""WBGene00022277""; transcript_id ""Y74C9..."


In [3]:
df_['feature'].value_counts()

exon               269868
CDS                222825
transcript          61073
gene                46742
stop_codon          33423
start_codon         33400
five_prime_utr      30958
three_prime_utr     27993
Name: feature, dtype: int64

In [4]:
kwargs_to_gtf = {'index': False, 'header': False, 'quoting': csv.QUOTE_NONE, 'sep': '\t'}
df_.to_csv(mp('canonical_geneset.gtf.gz'), compression='gzip', **kwargs_to_gtf)
df_.query('feature=="gene"').to_csv(mp('genes.gtf.gz'), compression='gzip', **kwargs_to_gtf)
df_.query('feature!="gene"').to_csv(mp('transcripts.gtf.gz'), compression='gzip', **kwargs_to_gtf)